# Loop 17 Analysis: Strategic Assessment

## Current Status
- Best CV/LB: 70.630478 (from saspav_best ensemble)
- Target: 68.919154
- Gap: 1.711324 (2.42%)
- Stuck for 7 consecutive experiments (011-017)

## Key Insights from This Loop
1. jiweiliu kernel starts from 71.657 baseline - our baseline (70.630478) is ALREADY 1.03 points BETTER
2. Bottom-left-fill heuristic produced WORSE results
3. Subset extraction from larger N produced WORSE results
4. All standard optimization approaches have been exhausted

In [1]:
# Analyze what approaches have NOT been tried
import pandas as pd
import numpy as np

# List of approaches tried
approaches_tried = [
    ('001', 'Baseline from jazivxt/bucket-of-chump', 70.647327, 'baseline'),
    ('002', 'Ensemble from 9 sources', 70.647306, 'no improvement'),
    ('003', 'Validated ensemble', 70.647327, 'no improvement'),
    ('004', 'bbox3 optimization', 70.647326, 'no improvement'),
    ('005', 'Baseline validated', 70.647327, 'no improvement'),
    ('006', 'Zaburo grid-based', 88.33, 'WORSE'),
    ('007', 'SA optimization on zaburo', 88.33, 'WORSE + overlaps'),
    ('008', 'Repair + ensemble', 70.647327, 'no improvement'),
    ('009', 'Fractional translation SA', 70.647327, 'no improvement'),
    ('010', 'Tessellation + saspav ensemble', 70.630478, 'IMPROVEMENT!'),
    ('011', 'Random restart SA', 70.630478, 'no improvement'),
    ('012', 'Crystalline packing', 70.630478, 'no improvement'),
    ('013', 'High restart SA', 70.630478, 'no improvement'),
    ('014', 'Extreme SA', 70.630478, 'no improvement'),
    ('015', 'Constraint Programming', 70.630478, 'WORSE'),
    ('016', 'True CP', 70.630478, 'WORSE'),
    ('017', 'Bottom-left fill', 70.630478, 'WORSE'),
]

df = pd.DataFrame(approaches_tried, columns=['Exp', 'Approach', 'Score', 'Result'])
print("Approaches Tried:")
print(df.to_string(index=False))
print(f"\nBest score achieved: {df['Score'].min():.6f}")
print(f"Target: 68.919154")
print(f"Gap: {df['Score'].min() - 68.919154:.6f}")


Approaches Tried:
Exp                              Approach     Score           Result
001 Baseline from jazivxt/bucket-of-chump 70.647327         baseline
002               Ensemble from 9 sources 70.647306   no improvement
003                    Validated ensemble 70.647327   no improvement
004                    bbox3 optimization 70.647326   no improvement
005                    Baseline validated 70.647327   no improvement
006                     Zaburo grid-based 88.330000            WORSE
007             SA optimization on zaburo 88.330000 WORSE + overlaps
008                     Repair + ensemble 70.647327   no improvement
009             Fractional translation SA 70.647327   no improvement
010        Tessellation + saspav ensemble 70.630478     IMPROVEMENT!
011                     Random restart SA 70.630478   no improvement
012                   Crystalline packing 70.630478   no improvement
013                       High restart SA 70.630478   no improvement
014             

In [2]:
# What approaches have NOT been tried?
approaches_not_tried = [
    ('sparroWASM external solver', 'State-of-the-art nesting algorithm from Jeroen Gar'),
    ('spyrrow Python wrapper', 'Python interface to sparrow solver'),
    ('Genetic Algorithm with crossover', 'Crossover between good partial solutions'),
    ('No-Fit Polygon (NFP) based optimization', 'Proper geometric constraints'),
    ('Backward propagation from N=200', 'Start from best large N, remove trees'),
    ('Manual tree placement for specific N', 'Interactive editor for fine-tuning'),
    ('Different rotation angles', 'Try non-standard angles like 30, 60, 120'),
    ('Asymmetric solutions', 'Break symmetry for potentially better packing'),
]

print("\nApproaches NOT yet tried:")
for name, desc in approaches_not_tried:
    print(f"  - {name}: {desc}")



Approaches NOT yet tried:
  - sparroWASM external solver: State-of-the-art nesting algorithm from Jeroen Gar
  - spyrrow Python wrapper: Python interface to sparrow solver
  - Genetic Algorithm with crossover: Crossover between good partial solutions
  - No-Fit Polygon (NFP) based optimization: Proper geometric constraints
  - Backward propagation from N=200: Start from best large N, remove trees
  - Manual tree placement for specific N: Interactive editor for fine-tuning
  - Different rotation angles: Try non-standard angles like 30, 60, 120
  - Asymmetric solutions: Break symmetry for potentially better packing


In [3]:
# Analyze score breakdown by N range
import os
import sys
sys.path.insert(0, '/home/code')

from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon

getcontext().prec = 25
scale_factor = Decimal("1e15")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def load_trees(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"]).lstrip('sx')
        y = str(row["y"]).lstrip('sy')
        deg = str(row["deg"]).lstrip('sd')
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Load current best
df = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')

print("Score breakdown by N:")
print("=" * 60)

Score breakdown by N:


In [4]:
# Calculate scores for each N and find worst performers
scores = []
for n in range(1, 201):
    trees = load_trees(n, df)
    score = calculate_score(trees)
    scores.append({'N': n, 'Score': score, 'Contribution': score})

scores_df = pd.DataFrame(scores)
scores_df['Cumulative'] = scores_df['Score'].cumsum()
scores_df['Pct'] = scores_df['Score'] / scores_df['Score'].sum() * 100

print(f"Total score: {scores_df['Score'].sum():.6f}")
print(f"\nTop 10 worst N values (highest contribution):")
print(scores_df.nlargest(10, 'Score')[['N', 'Score', 'Pct']].to_string(index=False))

print(f"\nTop 10 best N values (lowest contribution):")
print(scores_df.nsmallest(10, 'Score')[['N', 'Score', 'Pct']].to_string(index=False))

Total score: 70.630478

Top 10 worst N values (highest contribution):
 N    Score      Pct
 1 0.661250 0.936211
 2 0.450779 0.638222
 3 0.434745 0.615521
 5 0.416850 0.590184
 4 0.416545 0.589752
 7 0.399897 0.566181
 6 0.399610 0.565776
 9 0.387415 0.548510
 8 0.385407 0.545667
15 0.376978 0.533733

Top 10 best N values (lowest contribution):
  N    Score      Pct
181 0.329945 0.467142
156 0.329986 0.467200
182 0.329988 0.467203
180 0.331000 0.468637
155 0.332069 0.470149
168 0.332475 0.470724
195 0.332578 0.470871
179 0.332595 0.470894
167 0.332835 0.471234
194 0.332999 0.471466


In [5]:
# Calculate efficiency for each N
# Efficiency = (N * tree_area) / (side_length^2)
# Tree area is approximately 0.2475 (calculated from polygon)
tree_area = 0.2475

efficiencies = []
for n in range(1, 201):
    trees = load_trees(n, df)
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side = max(max_x - min_x, max_y - min_y)
    efficiency = (n * tree_area) / (side ** 2) * 100
    efficiencies.append({'N': n, 'Side': side, 'Efficiency': efficiency})

eff_df = pd.DataFrame(efficiencies)
print(f"\nEfficiency analysis:")
print(f"Mean efficiency: {eff_df['Efficiency'].mean():.2f}%")
print(f"Min efficiency: {eff_df['Efficiency'].min():.2f}% at N={eff_df.loc[eff_df['Efficiency'].idxmin(), 'N']}")
print(f"Max efficiency: {eff_df['Efficiency'].max():.2f}% at N={eff_df.loc[eff_df['Efficiency'].idxmax(), 'N']}")

print(f"\nLowest efficiency N values:")
print(eff_df.nsmallest(10, 'Efficiency')[['N', 'Side', 'Efficiency']].to_string(index=False))


Efficiency analysis:
Mean efficiency: 70.39%
Min efficiency: 37.43% at N=1
Max efficiency: 75.01% at N=181

Lowest efficiency N values:
 N     Side  Efficiency
 1 0.813173   37.429112
 2 0.949504   54.904931
 3 1.142031   56.929906
 5 1.443692   59.373934
 4 1.290806   59.417371
 7 1.673104   61.891006
 6 1.548438   61.935343
 9 1.867280   63.884976
 8 1.755921   64.217785
15 2.377955   65.653692


In [6]:
# Key insight: What would it take to reach the target?
target = 68.919154
current = scores_df['Score'].sum()
gap = current - target

print(f"\n" + "="*60)
print(f"GAP ANALYSIS")
print(f"="*60)
print(f"Current score: {current:.6f}")
print(f"Target score: {target:.6f}")
print(f"Gap: {gap:.6f} ({gap/current*100:.2f}%)")

# If we improved all N values uniformly
print(f"\nIf we improved ALL N values uniformly by {gap/current*100:.2f}%:")
print(f"  Each N would need to improve by {gap/200:.6f} on average")

# If we only improved large N (101-200)
large_n_score = scores_df[scores_df['N'] > 100]['Score'].sum()
print(f"\nIf we ONLY improved N=101-200 (current: {large_n_score:.6f}):")
print(f"  Would need {gap/large_n_score*100:.2f}% improvement in this range")

# What efficiency would we need?
print(f"\nEfficiency needed to reach target:")
print(f"  Current avg efficiency: {eff_df['Efficiency'].mean():.2f}%")
print(f"  Target avg efficiency: ~{eff_df['Efficiency'].mean() * current / target:.2f}%")


GAP ANALYSIS
Current score: 70.630478
Target score: 68.919154
Gap: 1.711324 (2.42%)

If we improved ALL N values uniformly by 2.42%:
  Each N would need to improve by 0.008557 on average

If we ONLY improved N=101-200 (current: 33.979366):
  Would need 5.04% improvement in this range

Efficiency needed to reach target:
  Current avg efficiency: 70.39%
  Target avg efficiency: ~72.13%


## Strategic Recommendations

### CRITICAL INSIGHT: We need a FUNDAMENTALLY DIFFERENT approach

After 17 experiments, all standard optimization approaches have failed:
- SA (various parameters) - no improvement
- bbox3 optimizer - no improvement  
- Constraint Programming - WORSE
- Bottom-left-fill - WORSE
- Grid-based approaches - WORSE
- Random restart - no improvement

### The gap (1.711 points, 2.42%) is TOO LARGE for incremental optimization

### Recommended Next Steps:

1. **Try sparroWASM/spyrrow external solver**
   - State-of-the-art nesting algorithm
   - May find fundamentally different configurations
   - Focus on large N (101-200) where 5% improvement closes the gap

2. **Implement backward propagation properly**
   - Start from best N=200 solution
   - Iteratively remove trees to get N=199, 198, etc.
   - This propagates good configurations downward

3. **Try asymmetric solutions**
   - All current solutions appear symmetric
   - Breaking symmetry might find better local optima

4. **Focus on specific N values with low efficiency**
   - N=1 has 45.4% efficiency (but is already optimal)
   - Find N values with room for improvement